In [1]:
# Datos: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-COVID-19.csv

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
# Loading the data
covid_bog = pd.read_csv( './data/covid19_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';' )

In [4]:
covid_bog.columns

Index(['ID de caso', 'Fecha de diagnóstico', 'Ciudad de residencia',
       'Localidad de residencia', 'Edad', 'Sexo', 'Tipo de caso', 'Ubicación',
       'Estado'],
      dtype='object')

In [5]:
covid_bog.shape

(27118, 9)

In [6]:
covid_bog[ 'Fecha de diagnóstico' ].replace( { '#¡REF!': None }, inplace = True )

In [7]:
covid_bog[ 'Fecha de diagnóstico' ] = pd.to_datetime( covid_bog[ 'Fecha de diagnóstico' ], format = '%d/%m/%y' )

In [8]:
covid_bog[ 'Localidad de residencia' ].replace( { 'Santa Fe': 'Santa Fé' }, inplace = True )

In [9]:
covid_bog.dtypes

ID de caso                          int64
Fecha de diagnóstico       datetime64[ns]
Ciudad de residencia               object
Localidad de residencia            object
Edad                                int64
Sexo                               object
Tipo de caso                       object
Ubicación                          object
Estado                             object
dtype: object

In [10]:
def get_age_group( x ):
    if x <= 19:
        return '0 - 19'
    elif x > 19 and x <= 39:
        return '20 - 39'
    elif x > 39 and x <= 59:
        return '40 - 59'
    elif x > 59 and x <= 79:
        return '60 - 79'
    elif x > 79:
          return '80+'
    else:
        return None

covid_bog[ 'Grupo de edad' ] = covid_bog[ 'Edad' ].apply( get_age_group )

In [11]:
covid_bog.rename( columns = { 'ID de caso': 'ID_caso', 'Fecha de diagnóstico': 'Fecha_diagnostico', 'Ciudad de residencia': 'Ciudad', 'Localidad de residencia': 'Localidad', 'Tipo de caso': 'Tipo', 'Ubicación': 'Atencion', 'Grupo de edad': 'Grupo_de_edad' }, inplace = True )

In [12]:
covid_bog.dtypes

ID_caso                       int64
Fecha_diagnostico    datetime64[ns]
Ciudad                       object
Localidad                    object
Edad                          int64
Sexo                         object
Tipo                         object
Atencion                     object
Estado                       object
Grupo_de_edad                object
dtype: object

In [13]:
covid_bog.tail()

,ID_caso,Fecha_diagnostico,Ciudad,Localidad,Edad,Sexo,Tipo,Atencion,Estado,Grupo_de_edad
27113,27114,2020-06-26,Bogotá,Sin Dato,9,M,En estudio ...,Casa,Moderado,0 - 19
27114,27115,2020-06-26,Bogotá,Sin Dato,48,M,En estudio ...,Casa,Moderado,40 - 59
27115,27116,2020-06-26,Bogotá,Sin Dato,28,F,En estudio ...,Casa,Moderado,20 - 39
27116,27117,2020-06-27,Bogotá,Sin Dato,43,M,En estudio ...,Casa,Moderado,40 - 59
27117,27118,2020-06-27,Bogotá,Sin Dato,33,F,En estudio ...,Casa,Moderado,20 - 39


In [14]:
covid_bog[ 'Estado' ].unique()

array(['Recuperado', 'Fallecido', 'Crítico', 'Moderado', 'Severo'],
      dtype=object)

In [15]:
covid_bog.to_csv( './data/covid19_bogota.csv', index = False )